# Loyalty Scheme Analysis

## Table of Contents
1. [Introduction](#1.-Introduction)
2. [Data Loading](#2.-Data-Loading)
4. [Exploratory Data Analysis (EDA)](#3.-Exploratory-Data-Analysis-(EDA))
5. [Hypothesis Testing](#4.-Hypothesis-Testing)
6. [Conclusions](#5.-Conclusions)

---

## 1. Introduction
**Purpose:** The purpose of this notebook is to analyse Echo Circuit's loyalty scheme purchases, focusing on determining whether purchases made with a loyalty memebrship and purchases made without a loyalty memebrship exhibit similar transaction value distributions. Therefore, revealing insights into loyalty members' behaviour.

This analysis is part of my analysis of Echo Circuit's 2019 sales performance. For the complete report please read the [README.md](https://github.com/aranjeetpaul/Echo-Circuit-2019-sales-analysis/blob/main/README.md) from my Github repository.

**Dataset Overview**: This project uses the `cleaned_data.xlsx` dataset which can be found [here](https://github.com/aranjeetpaul/Echo-Circuit-2019-sales-analysis/blob/main/Data/cleaned_data.xlsx). This dataset contains individual order records, including order value, loyalty status, month of order, region of purchase, etc. Original raw data can be found [here](https://github.com/aranjeetpaul/Echo-Circuit-2019-sales-analysis/blob/main/Data/raw_data.xlsx) and SAS/SQL code used to clean it can be found [here](https://github.com/aranjeetpaul/Echo-Circuit-2019-sales-analysis/blob/main/Data%20Cleaning%20%26%20Analysis/Data%20Cleaning.sas).

---

## 2. Data Loading
Load the `cleaned_data.xlsx` dataset. Then create tables required for analysis:
- `loyalty` dataset containing values of all loyalty purchases.
- `non-loyalty` dataset containing values of all non-loyalty purchases.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import shapiro, kstest, levene, mannwhitneyu, skew, kurtosis

# Apply the Seaborn style
plt.style.use('seaborn-v0_8-darkgrid')

### 2.1 Load Data

Load `cleaned_data.xlsx` dataset.

In [ ]:
# Load data
df = pd.read_excel('cleaned_data.xlsx')
# Check all rows and columns are loaded
print(df.shape)
df.head()

### 2.2 Create Tables Required For Analysis

Create `loyalty` dataset containing values of transactions by loyalty members and `non-loyalty` datasets containing values of transactions by non-loyalty members.

In [ ]:
# Create seperate tables for the value of transactions of loyalty and no loyalty purchases
loyalty = df[df['LOYALTY'] == 'Y']['VALUE']
non_loyalty = df[df['LOYALTY'] == 'N']['VALUE']

---

## 3. Exploratory Data Analysis (EDA)

Exploratory Data Analysis (EDA) provides an initial understanding of `loyalty` and `non-loyalty` datasets' underlying distributions and patterns.

### 3.1 Initial Analysis

From the below analysis we see that `loyalty` and `non-loyalty` datasets are similar in count, mean, standard deviation, minimum value and quartiles. But different in maximum value, skewness and kurtosis. Suggesting the existence of outliers in the `loyalty` dataset that the `non-loyalty` dataset may not have.

In [ ]:
# Gather basic statistics of lyalty purchases
print('Description of loyalty purchases value:')
print(loyalty.describe())
print('skewness:', skew(loyalty))
print('kurtosis:', kurtosis(loyalty), '\n')

# Gather basic statistics of no lyalty purchases
print('Description of non-loyalty purchases value:')
print(non_loyalty.describe())
print('skewness:', skew(non_loyalty))
print('kurtosis:', kurtosis(non_loyalty))

### 3.2 Distribution Visualisation

Below you can see Q-Q plots, boxplots and histograms for both `loyalty` and `non-loyalty` datasets. We can see both are highly skewed and the `loyalty` dataset have outliers with a value of over £500.

In [ ]:
# Create a Q-Q plot of loyalty purchases
sm.qqplot(loyalty, line='45')
plt.title('Q-Q Plot of Loyalty Purchases')
plt.show()
# Create a Boxplot and Histogram of loyalty purchases
f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)})
sns.boxplot(loyalty, orient="h", ax=ax_box)
sns.histplot(data=loyalty, ax=ax_hist)
ax_box.set(xlabel='')
ax_hist.set(xlabel='Transaction Value', ylabel='Fequency')
plt.suptitle('Boxplot and Histogram of Loyalty Purchases')
plt.show()

# Create a Q-Q plot of no loyalty purchases
sm.qqplot(non_loyalty, line='45')
plt.title('Q-Q Plot of No Loyalty Purchases')
plt.show()
# Create a Boxplot and Histogram of loyalty purchases
f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)})
sns.boxplot(non_loyalty, orient="h", ax=ax_box)
sns.histplot(data=non_loyalty, ax=ax_hist)
ax_box.set(xlabel='')
ax_hist.set(xlabel='Transaction Value', ylabel='Fequency')
plt.suptitle('Boxplot and Histogram of Non-Loyalty Purchases')
plt.show()

### 3.3 Repeat Purchases of Loyalty Members

Below is a table showing the number of repeat purchases of customers with a loyalty memberships. We can see most Loyalty members don't carry out repeat purchases.

In [ ]:
# Collect Loyalty ID's
loyalty_ids = df[df['LOYALTY'] == 'Y']['LOYALTY_ID']
# Calculate frequency table
freq_tab = pd.DataFrame(loyalty_ids.value_counts().value_counts())
# Rename index and column
freq_tab = freq_tab.rename(columns={'count':'Frequency'})
freq_tab = freq_tab.rename_axis(None, axis=0)
# Rename index to make more sense
freq_tab = freq_tab.rename(index={1:'No repeat purchases', 2:'1 repeat purchases', 3:'2 repeat purchases', 4:'3 repeat purchases'})
# Calculate column containing percentage of loyalty purchases
freq_tab['% of All Loyalty Purchases'] = (freq_tab['Frequency'] / sum(freq_tab['Frequency']))
freq_tab['% of All Loyalty Purchases'] = freq_tab['% of All Loyalty Purchases'].map('{:.1%}'.format)
freq_tab

---

## 4. Hypothesis Testing

Perform hypothesis tests to compare `loyalty` and `non-loyalty` datasets.

### 4.1 Shapiro-Wilk Test

Using the Shapiro-Wilk test below we can see that for both the `loyalty` and `non-loyalty` datasets we have sufficient evidence to say neither are normally distributed. This  result is useful for telling us about the datasets and will also be used to determine which hypothsis tests we can use later on.

In [ ]:
# Run Shapiro-Wilk test of normality for loyalty and no loyalty purchases
sha_stat_loy, sha_pval_loy = shapiro(loyalty)
sha_stat_non_loy, sha_pval_non_loy = shapiro(non_loyalty)

# Set significane level to 0.5
alpha = 0.05
# Print conclusions
if sha_pval_loy < alpha:
    print('P-value for loyalty purchases Shapiro-Wilk test:', sha_pval_loy)
    print('So we have sufficient evidence to say that the loyalty dataset does not come from a normal distribution.')
else:
    print('P-value for loyalty purchases Shapiro-Wilk test:', sha_pval_loy)
    print('So we have insufficient evidence to say that the loyalty dataset does not come from a normal distribution.')

if sha_pval_non_loy < alpha:
    print('P-value for non-loyalty purchases Shapiro-Wilk test:', sha_pval_non_loy)
    print('So we have sufficient evidence to say that the non-loyalty dataset does not come from a normal distribution.')
else:
    print('P-value for non-loyalty purchases Shapiro-Wilk test:', sha_pval_non_loy)
    print('So we have insufficient evidence to say that the non-loyalty dataset does not come from a normal distribution.')


### 4.2 Levene’s Test

Using the Levene’s test below we can see that we have insufficient evidence to say that the variances of the `loyalty` and `non-loyalty` datasets are significantly different.

In [ ]:
# Run Levene's test
lev_stat, lev_pval = levene(loyalty, non_loyalty, center='median')

# Set significane level to 0.5
alpha = 0.05
# Print conclusions
if lev_pval < alpha:
    print('P-value for Levene\'s test:', lev_pval)
    print('So we have sufficient evidence to say that the variances ofthe loyalty and non-loyalty datasets are significantly different.')
else:
    print('P-value for Levene\'s test:', lev_pval)
    print('So we have insufficient evidence to say that the variances of the loyalty and non-loyalty datasets are significantly different.')

### 4.3 Mann-Whitney U Test

Using the Mann-Whitney U test below we can see that we have insufficient evidence to say that the means of the `loyalty` and `non-loyalty` are significantly different.

In [ ]:
# Run Mann-Whitney U Test
mann_stat, mann_pval = mannwhitneyu(loyalty, non_loyalty)

# Set significane level to 0.5
alpha = 0.05
# Print conclusions
if mann_pval < alpha:
    print('P-value for Mann-Whitney U test:', mann_pval)
    print('So we have sufficient evidence to say that the means of the loyalty and non-loyalty datasets are significantly different.')
else:
    print('P-value for Mann-Whitney U test:', mann_pval)
    print('So we have insufficient evidence to say that the means of the loyalty and non-loyalty datasets are significantly different.')

---

## 5. Conclusions

From the above analysis, we can draw 2 main conclusions:

- Most loyalty members don't carry out repeat purchases.
- Both loyalty and non-loyalty members exhibit similar behaviours.

Unfortnately, this suggests Echo Circuits loyalty scheme is a failure in encouraging sales growth.

For a complete report of Echo Circuit's 2019 sale performance, please read the [README.md](https://github.com/aranjeetpaul/Echo-Circuit-2019-sales-analysis/blob/main/README.md) from my Github repository.